### Dagshub Tutorial

In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8,
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

### Handle class imbalance

In [4]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619], dtype=int64))

### Track Experiments

In [5]:
models = [
    (
        "Logistic Regression",
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest",
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(),
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [6]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]

    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [7]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [8]:
#dagshub setup
import dagshub
dagshub.init(repo_owner='harsh.arya1004', repo_name='Mlflow_tutorial', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=3f230ba4-5b1a-4084-802e-41e68e1c8ac8&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=3ba81b29fe3753b7bee328ba9b78f64173d71f785df2a526f25a489494e385a4




Accessing as harsh.arya1004

Initialized MLflow to track repo "harsh.arya1004/Mlflow_tutorial"

Repository harsh.arya1004/Mlflow_tutorial initialized!

In [10]:
mlflow.set_experiment("Anomaly Detection")
mlflow.set_tracking_uri("https://dagshub.com/harsh.arya1004/Mlflow_tutorial.mlflow")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]

    metrics = dict()
    metrics['accuracy'] = reports[i]['accuracy']
    metrics['recall_class_0'] = reports[i]['0']['recall']
    metrics['recall_class_1'] = reports[i]['1']['recall']
    metrics['f1_score_macro'] = reports[i]['macro avg']['f1-score']
    metrics['f1_score_weighted'] = reports[i]['weighted avg']['f1-score']

    with mlflow.start_run(run_name=model_name):
        mlflow.log_params(params)
        mlflow.log_metrics(metrics)
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, 'model')
        else:
            mlflow.sklearn.log_model(model, 'model')

### Register the model(On Remote - Dagshub)

In [19]:
model_name = "XGBClassifier"
run_id = input("Enter the run id for this run: ")
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri , model_name)

Registered model 'XGBClassifier' already exists. Creating a new version of this model...
2025/06/25 16:56:37 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGBClassifier, version 2
Created version '2' of model 'XGBClassifier'.


<ModelVersion: aliases=[], creation_timestamp=1750850797376, current_stage='None', description='', last_updated_timestamp=1750850797376, name='XGBClassifier', run_id='d962619620e24edfb5b6739a5faa8c82', run_link='', source='mlflow-artifacts:/1fa05a5e82b8460aa6c0908366774c26/d962619620e24edfb5b6739a5faa8c82/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>

### Transition from Model to Production - (On Remote)

In [21]:
from mlflow import MlflowClient
current_model_uri = f"models:/{model_name}/2"
prod_model_name = 'final-model'

client = MlflowClient()

client.copy_model_version(src_model_uri=current_model_uri, dst_name=prod_model_name)

<ModelVersion: aliases=[], creation_timestamp=1750851084890, current_stage='None', description='', last_updated_timestamp=1750851084890, name='final-model', run_id='d962619620e24edfb5b6739a5faa8c82', run_link='', source='models:/XGBClassifier/2', status='READY', status_message='', tags={}, user_id='', version='2'>

### Loading the Production Model - (From Remote)

In [24]:
model_name = 'final-model'
model_uri = f"models:/{model_name}@champion"
loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:5]

array([0, 0, 0, 0, 0])